In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers

import warnings
warnings.filterwarnings(
  'ignore'
)

In [38]:
df = pd.DataFrame({
    "soil_moisture": [0.10, 0.15, 0.20, 0.25, 0.40, 0.60, 0.35, 0.18,
                      0.45, 0.05, 0.80, 0.27, 0.55, 0.70, 0.12, 0.30],
    
    "temperature_c": [34, 30, 26, 22, 28, 30, 19, 22,
                      35, 24, 33, 33, 21, 25, 20, 29],
    
    "sunlight_hours": [9, 8, 7, 4, 8, 10, 3, 10,
                       12, 5, 9, 11, 2, 6, 1, 9],
    
    "needs_water": [1, 1, 1, 0, 0, 0, 0, 1,
                    0, 1, 0, 1, 0, 0, 1, 1]
})

df

,soil_moisture,temperature_c,sunlight_hours,needs_water
0,0.10,34,9,1
1,0.15,30,8,1
2,0.20,26,7,1
3,0.25,22,4,0
4,0.40,28,8,0
5,0.60,30,10,0
6,0.35,19,3,0
7,0.18,22,10,1
8,0.45,35,12,0
9,0.05,24,5,1


In [39]:
X = df[['soil_moisture', 'temperature_c', 'sunlight_hours']]
y = df['needs_water']
X

,soil_moisture,temperature_c,sunlight_hours
0,0.10,34,9
1,0.15,30,8
2,0.20,26,7
3,0.25,22,4
4,0.40,28,8
5,0.60,30,10
6,0.35,19,3
7,0.18,22,10
8,0.45,35,12
9,0.05,24,5


In [40]:
Scaler = MinMaxScaler()

X_scaled = pd.DataFrame(
  Scaler.fit_transform(X),
  columns=X.columns,
  index=df.index
)
X_scaled

,soil_moisture,temperature_c,sunlight_hours
0,0.066667,0.9375,0.727273
1,0.133333,0.6875,0.636364
2,0.200000,0.4375,0.545455
3,0.266667,0.1875,0.272727
4,0.466667,0.5625,0.636364
5,0.733333,0.6875,0.818182
6,0.400000,0.0000,0.181818
7,0.173333,0.1875,0.818182
8,0.533333,1.0000,1.000000
9,0.000000,0.3125,0.363636


In [41]:
X_train, X_test, y_train, y_test = train_test_split(
  X_scaled,
  y,
  test_size=0.2,
  random_state=42
)
X_train.shape, X_test.shape

((12, 3), (4, 3))

In [42]:
print(tf.config.list_physical_devices('GPU'))

[]


In [43]:
model = keras.Sequential(
  [
    layers.Input(shape=(X_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
  ]
)

In [44]:
model.compile(
    optimizer='sgd',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [45]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,481 (17.50 KB)

 Trainable params: 4,481 (17.50 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
history = model.fit(
  X_train.values,
  y_train.values,
  validation_data = (X_test.values, y_test.values),
  epochs = 100,
  batch_size = 4,
  verbose = 1
)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.7500 - loss: 0.5772 - val_accuracy: 0.7500 - val_loss: 0.6510
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.7500 - loss: 0.5741 - val_accuracy: 0.7500 - val_loss: 0.6499
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.7500 - loss: 0.5714 - val_accuracy: 0.7500 - val_loss: 0.6490
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.7500 - loss: 0.5704 - val_accuracy: 0.7500 - val_loss: 0.6482
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.7500 - loss: 0.5702 - val_accuracy: 0.7500 - val_loss: 0.6471
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.7500 - loss: 0.5680 - val_accuracy: 0.7500 - val_loss: 0.6463
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.8333 - loss: 0.5677 - val_accuracy: 0.7500 - val_loss: 0.6455
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7500 - loss: 0.5657 - val_accuracy: 0.7500 - va